# Chain-of-Verification GPT4 Template
Chain-of-Verification (CoVe) is a prompt engineering technique to bypass hallucinations. An LLM generates a baseline response to a user query, but this might contain errors. CoVe helps by creating a plan comprising of verification questions that are used to validate the information. This process results in more accurate answers than the initial response. The final answer is revised based on these validations. **[ Link to Paper](https://arxiv.org/pdf/2309.11495.pdf)**

**How to use template?**
1. Download `cove_template_config.json` (add github link)
2. Follow instructions in this notebook.

*Need a playground to test your prompts?*
Go to this [AI Workbook](https://lastmileai.dev/workbooks/clpa6nv2x00fdqp38oefmz29c) to experiment with your prompts/model params.

In [ ]:
# Install AIConfig package
!pip install python-aiconfig

In [59]:
# Import required modules from AIConfig and other dependencies
import openai
import pandas as pd
from google.colab import userdata
from aiconfig import AIConfigRuntime, CallbackManager, InferenceOptions

# Use your OpenAI Key
openai.api_key = userdata.get('openai_key')

In [60]:
# First, upload `cove_template_config.json`to Files in Colab Notebook
# Load the config.
config = AIConfigRuntime.load("cove_template_config.json")
config.callback_manager = CallbackManager([])

# Setup for streaming responses from LLM
inference_options = InferenceOptions()

## 1. Baseline Response
Prompt LLM with user question. The baseline response from the LLM might contain inaccuracies that we will want to verify. The user prompt is from AIConfig named
`baseline_response_gen`.

**Prompt: Name 15 politicians who were born in New York City, New York.**

In [61]:
# TODO: Update baseline_prompt but ensure it is structured in a way that outputs a list of entities where each can be verified.
baseline_prompt = "Name 15 politicians who were born in New York City, New York"

async def run_baseline_prompt(baseline_prompt):
    """
    Run GPT4 baseline prompt to generate a list of entities and retrieve the response.

    :param baseline_prompt: User query for baseline prompt (runs with GPT4)
    :return: The output text from the generated baseline response.
    """
    params = {"baseline_prompt": baseline_prompt}

    await config.run("baseline_response_gen", params, inference_options)
    return config.get_output_text("baseline_response_gen")

baseline_reponse = await run_baseline_prompt(baseline_prompt)

1. Theodore Roosevelt - 26th President of the United States
2. Franklin D. Roosevelt - 32nd President of the United States
3. Donald Trump - 45th President of the United States
4. Michael Bloomberg - Former Mayor of New York City
5. Rudy Giuliani - Former Mayor of New York City
6. Alexander Hamilton - Founding Father of the United States
7. Eliot Spitzer - Former Governor of New York
8. Chuck Schumer - U.S. Senator from New York
9. Kirsten Gillibrand - U.S. Senator from New York
10. Bill de Blasio - Former Mayor of New York City
11. Andrew Cuomo - Former Governor of New York
12. Eric Schneiderman - Former Attorney General of New York
13. Letitia James - Attorney General of New York
14. Alexandria Ocasio-Cortez - U.S. Representative from New York
15. Adam Clayton Powell Jr. - Former U.S. Representative from New York


## 2. Setup and Test Verification Question
Given both query and baseline response, generate a verification
question that could help to self-analyze if there are any mistakes in the original response. We will use one verification question here. The verification prompt is from AIConfig, named `verification`.

**Verification Prompt: Where was {{entity}} born?**

In [62]:
# TODO: Update verification question that takes in entity as a parameter
verification_question = "Where was {{entity}} born"

async def run_single_verification(verification_question, entity):
    """
    Runs GPT4 single verification prompt and returns the response.

    :param verification_question: The question to verify each entity from the baseline response.
    :param entity: The single entity to be verified.
    :return: Response from the LLM.
    """
    params = {
        "verification_question": verification_question,
        "entity": entity
    }

    # Disables irrelevant chat context to avoid context limits
    config.set_metadata("remember_chat_context", False, "verification")

    verification_completion = await config.run("verification", params, options=inference_options)

    return verification_completion

# Test verification question on a single entity
verification_completion = await run_single_verification(verification_question, "Franklin D. Roosevelt")

Franklin D. Roosevelt was born in Hyde Park, New York, USA.

## 3. Execute Verifications
Answer each verification question for each entity from the the baseline response. Save the verification results to be used to generate the revised response.

In [63]:
def gen_entities_list(baseline_response):
  """
  Extracts entity names from a given baseline response by processing each line with regex.
  TODO: Update regex if the format of the baseline response changes. (ex. not a numbered list)

  :param baseline_response: A string of newline-separated entity records.
  :return: A list of extracted entity names.
  """
  rows = baseline_response.split('\n')
  entities = []

  for row in rows:
      if not row.strip():
          continue
      entities.append(pd.Series(row).str.extract(r'(\d+\.\s)([^,]*)')[1].values[0])

  return entities

async def gen_verification_results(entities):
  """
  Runs GPT4 verification prompt for each entity and concatenates returned verifications into a single string.

  :param entities: List of entity names to verify.
  :return: String containing all verification results.
  """
  verification_data = ""
  for n in entities:
      params = {"entity": n}
      verification_completion = await config.run("verification", params, options=inference_options)
      single_verification_text = config.get_output_text("verification")
      verification_data += " " + single_verification_text
      print("\n")

  return single_verification_text


entities = gen_entities_list(baseline_response)
verification_data = await gen_verification_results(entities)

Theodore Roosevelt was born in New York City, New York, USA.

Franklin D. Roosevelt was born in Hyde Park, New York, USA.

Donald Trump was born in Queens, New York, USA.

Michael Bloomberg was born in Boston, Massachusetts, USA.

Rudy Giuliani was born in Brooklyn, New York, USA.

Chuck Schumer was born in Brooklyn, New York, USA.

Kirsten Gillibrand was born in Albany, New York, USA.

Alexandria Ocasio-Cortez was born in The Bronx, New York City, USA.

Eliot Spitzer was born in The Bronx, New York, USA.

Andrew Cuomo was born in Queens, New York, USA.

Bill de Blasio was born in Manhattan, New York, USA.

Eric Adams was born in Brownsville, Brooklyn, New York City, USA.

Jerrold Nadler was born in Brooklyn, New York, USA.

Carolyn Maloney was born in Greensboro, North Carolina, USA.

Letitia James was born in Brooklyn, New York, USA.



## 4. Generate Revised Response
Given the discovered inconsistencies (if any), generate a revised response incorporating the verification results.

In [64]:
# Disable irrelevant chat context to avoid context limits
config.set_metadata("remember_chat_context", False, "final_response_gen")

# Generated the revised response using verification data
params = {"verification_results": verification_data}
revised_response = await config.run("final_response_gen", params, options=inference_options)

### Revised Response 
1. Theodore Roosevelt - 26th President of the United States
2. Franklin D. Roosevelt - 32nd President of the United States
3. Donald Trump - 45th President of the United States
4. Michael Bloomberg - Former Mayor of New York City
5. Rudy Giuliani - Former Mayor of New York City
6. Alexander Hamilton - Founding Father of the United States
7. Eliot Spitzer - Former Governor of New York
8. Chuck Schumer - U.S. Senator from New York
9. Kirsten Gillibrand - U.S. Senator from New York
10. Bill de Blasio - Former Mayor of New York City
11. Andrew Cuomo - Former Governor of New York
12. Eric Schneiderman - Former Attorney General of New York
13. Alexandria Ocasio-Cortez - U.S. Representative from New York
14. Adam Clayton Powell Jr. - Former U.S. Representative from New York

### Failed Entities 
Letitia James - Attorney General of New York. Reason: The verification data shows that Letitia James was born in Brooklyn, New York, not New York City, New York.